In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ca = pd.read_csv('../data/Initiated Building Permits (CA Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
fr = pd.read_csv('../data/Initiated Building Permits (FR Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pl = pd.read_csv('../data/Initiated Building Permits (PL Tasks) 3yr.csv',
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
pw = pd.read_csv('../data/Initiated Building Permits (PW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
sw = pd.read_csv('../data/Initiated Building Permits (SW Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
ws = pd.read_csv('../data/Initiated Building Permits (WS Tasks) 3yr.csv', 
                 parse_dates=['Date_Initiated','Date_Issued','Task_Complete_Date']) 
u_o = pd.read_csv('../data/Initiated Building Permits 3yr (Temp-Final U_O).csv', 
                  parse_dates=['Date_Initiated','Date_Issued','Temp U&O Complete Date','Final U&O Complete Date']) 

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#from Patrick
department = [['Codes(CA)', ca, 'ca'],
              ['Fire Marshal(FR)', fr, 'fr'],
              ['Planning(PL)', pl, 'pl'],
              ['Public Works(PW)', pw, 'pw'],
              ['Storm Water (SW)', sw, 'sw'],
              ['Water Sewer(WS)', ws, 'ws'] ]
df_list = list()

for a,b,c in department:
    ct = b[['CASE_TYPE', 'CASE_TYPE_DESC']].copy()
    ct['DEPARTMENT'] = a
    ct['dept'] = c
    ct_clean = ct.value_counts().reset_index().drop(columns=[0])
    df_list.append(ct_clean)
ct_df = pd.concat(df_list).sort_values(by =['CASE_TYPE', 'DEPARTMENT'])
case_df = ct_df.iloc[:,0:2].drop_duplicates().reset_index()

In [4]:
dept_df = ct_df.iloc[:,2:5].drop_duplicates().reset_index()

In [5]:
list_dept = {'fr':fr,'pl':pl,'pw':pw,'sw':sw,'ws':ws,'ca':ca}
convert_dict = {'CASE_TYPE': 'category',
                'SUB_TYPE': 'category',
                'STATUS_CODE': 'category',
                'RESULT_CODE': 'category'
               }
all_depts_app = pd.DataFrame()
all_depts_inspect = pd.DataFrame()
task_filter = {}
all_pivot = pd.DataFrame()

In [6]:
for key in list_dept:
    df = list_dept[key]
    df['dept'] = str(key)
    df = df[df['PERMIT #'].astype(str).str.isnumeric()] # T&D filter
    df = df.astype(convert_dict)
    df['deltatask'] = df['Task_Complete_Date'] - df['Date_Initiated'] #create column for task length
    df = df.astype({'deltatask':"timedelta64[D]"}) #make timedelta work for pivot table
    # find tasks completed after permit issued
    df['postissuedtask'] = df['Task_Complete_Date'] > df['Date_Issued']
    post_taskcode = df[['TASK_DESC','postissuedtask']].groupby(['TASK_DESC']).agg(['sum', 'count'])
    filter_posttaskcode = post_taskcode[post_taskcode.iloc[:,0] >= post_taskcode.iloc[:,1]*.95].reset_index()
    #create list of tasks for application and inspection 
    df_inspect = df[df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    df_app = df[~df['TASK_DESC'].isin(filter_posttaskcode.TASK_DESC)]
    #keep only the last task completed
    df_last = df_app.sort_values(by=['Task_Complete_Date','RESULT_DESC'], ascending=[False,True]).drop_duplicates(subset=['PERMIT #', 'TASK_CODE'], keep='first')
    #pivot data to aggregate
    df_pivot = pd.pivot_table(df_last, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
    df_pivot['dept'] = str(key)
    #save all the steps
    all_depts_app = all_depts_app.append(df_last)
    all_depts_inspect = all_depts_inspect.append(df_inspect)
    task_filter.update({str(key):filter_posttaskcode['TASK_DESC'].to_list()})
    all_pivot = all_pivot.append(df_pivot)
    print(key)
    
    

fr
pl
pw
sw
ws
ca


In [7]:
task_filter

{'fr': ['CA - U&O Fire Sprinkler Final Approval',
  'Fire Alarm Review On Bldg App',
  'Fire Sprinkler RTC Review - CA Return to Customer',
  'Fire Sprinkler Review On Bldg App',
  'Fire Sprinkler Review on Alarm App',
  'Life Safety Rough In',
  'U&O Alarm Fire Sprinkler Final Approval',
  'U&O Fire Alarm System Final Approval - CA',
  'U&O Fire Alarm System Final Approval - CAAF',
  'U&O Life Safety Final Approval',
  'U&O Underground Work Final Approval - CA',
  '[B]  Fire Alarm System Review',
  '[B]  Fire Sprinkler Review On Bldg App',
  '[B] Fire Alarm Review On Bldg App',
  '[B] Fire Underground Work Review On Bldg App'],
 'pl': [],
 'pw': ['CA - U&O Public Works Final Approval',
  'U&O PW ROW FA - CA Final Approval',
  'U&O PW Ramp & Curb Cut FA - CA Final Approval',
  'U&O PW Ramp RTC & Cut - CA Return to Customer',
  'U&O PW Sidewalk FA - CA Final Approval',
  'U&O PW Solid Waste FA - CA Final Approval'],
 'sw': ['SWGR U&O Sign-off', 'U&O SW Flood Plain Final Approval - CA'],

In [8]:
all_depts_app.RESULT_DESC.value_counts()

Approved                                   128918
Approved with Comments                      48947
No sign-off needed on this project          41071
Issued                                      27004
OK to Pay Permit Fee                        25649
Not Applicable                              25253
Yes                                         12386
No                                           5128
Sidewalks Required                           4689
Paper plans received                         3309
Electronic plans received                    3135
No Capital Project Planned                   1812
Sidewalks Required - In Lieu Elligble        1368
Sidewalks not required                       1279
Allowed and elected to pay in lieu           1217
Elected/Required to construct sidewalks       290
Plans received via email as a PDF             226
Granted with Conditions                       119
BZA Variance Applied for                       85
Zoning Waiver Requested                        77


In [9]:
alltemppivot = pd.pivot_table(all_depts_app, values='deltatask', index=['CASE_TYPE'], aggfunc=[np.mean, np.median, np.min, np.max, np.std, np.var], margins = True)
alltemppivot['dept'] = str('all')
all_pivot = all_pivot.append(alltemppivot)
all_pivot = all_pivot.reset_index()

In [10]:
taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })

<ipython-input-10-a55de755209d>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  taskfilterdf = pd.DataFrame({ key:pd.Series(value) for key, value in task_filter.items() })


In [11]:
#Adjust columns for readability
new_columns = {'CASE_TYPE': 'CASE_TYPE', 'meandeltatask' : 'MEAN', 
               'mediandeltatask': 'MEDIAN',
               'amindeltatask' : 'AMIN' ,
               'amaxdeltatask' : 'AMAX',
               'stddeltatask': 'STD',
              'vardeltatask': 'VAR',
               'dept': 'dept'}

all_pivot.columns = all_pivot.columns.map(''.join)
all_pivot.rename(columns = new_columns, inplace=True)
all_pivot.columns

Index(['CASE_TYPE', 'MEAN', 'MEDIAN', 'AMIN', 'AMAX', 'STD', 'VAR', 'dept'], dtype='object')

In [12]:
all_pivot_label = all_pivot.copy()
column = []
for x in all_pivot['dept']:
    if str(x) == 'all':
        column.append('All')
    else:
        column.append(dept_df.at[dept_df.loc[dept_df['dept'] == str(x)].index[0],'DEPARTMENT'])
all_pivot_label['DEPARTMENT'] = column
column2 = []
for x in all_pivot['CASE_TYPE']:
    if str(x) not in case_df['CASE_TYPE'].to_list():
        column2.append('All')
    else:
        column2.append(case_df.at[case_df.loc[case_df['CASE_TYPE'] == str(x)].index[0],'CASE_TYPE_DESC'])
    
all_pivot_label['Case Description'] = column2
all_pivot_label = all_pivot_label.drop(['dept'], axis=1)
all_pivot_label

,CASE_TYPE,MEAN,MEDIAN,AMIN,AMAX,STD,VAR,DEPARTMENT,Case Description
0,CACA,59.356234,30.0,0.0,529.0,80.371588,6459.592161,Fire Marshal(FR),Building Commercial - Addition
1,CACF,34.920635,15.0,0.0,385.0,58.886835,3467.659286,Fire Marshal(FR),Building Commercial - Foundation
2,CACG,58.962264,48.0,16.0,198.0,44.781075,2005.344702,Fire Marshal(FR),Building Commercial - Structural Frame
3,CACH,95.737885,71.0,0.0,604.0,94.584601,8946.246818,Fire Marshal(FR),Building Commercial - Shell
4,CACJ,27.380282,1.0,0.0,152.0,47.071486,2215.724748,Fire Marshal(FR),Building Commercial - Fire Damage
...,...,...,...,...,...,...,...,...,...
166,CARP,7.824885,0.0,0.0,124.0,16.301907,265.752174,All,Building Residential - Change Contractor
167,CARR,5.649149,0.0,-30.0,574.0,18.484825,341.688740,All,Building Residential - Rehab
168,CARS,19.288732,10.0,0.0,158.0,26.251133,689.121988,All,Building Residential New Storm Damage
169,CART,66.496190,27.0,0.0,474.0,87.116068,7589.209232,All,Building Residential - Tenant Finish Out


In [13]:
#clean up the u_o sheet to match
u_o['postissuedtask'] = True
u_o['phase']= 'inspection'
u_o['Task_Complete_Date'] = u_o['Final U&O Complete Date']
u_o = u_o.dropna(subset=['Final U&O Complete Date'])
u_o['deltatask'] = u_o['Task_Complete_Date'] - u_o['Date_Initiated'].copy()
u_o = u_o.astype({'deltatask':"timedelta64[D]"})
u_o

,PERMIT #,CASE_TYPE,CASE_TYPE_DESC,SUB_TYPE,SUB_TYPE_DESC,STATUS_CODE,Date_Initiated,Date_Issued,TASK_CODE,TASK_DESC,Temp U&O Complete Date,Final U&O Complete Date,postissuedtask,phase,Task_Complete_Date,deltatask
0,2018000034,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2018-11-26,True,inspection,2018-11-26,328.0
1,2018000187,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2020-05-05,True,inspection,2020-05-05,854.0
2,2018045480,CACN,Building Commercial - New,CAD02B016,"General Office, Professional Services",DONE,2018-08-02,2019-01-02,CABUO,U&O Letter,2020-05-12,2020-07-27,True,inspection,2020-07-27,725.0
3,2018057638,CACR,Building Commercial - Rehab,CAF31A203,Restaurant (Take Out),DONE,2018-09-10,2019-01-02,CABUO,U&O Letter,NaT,2019-11-14,True,inspection,2019-11-14,430.0
4,2018059485,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",DONE,2018-09-18,2019-01-02,CABUO,U&O Letter,NaT,2019-11-05,True,inspection,2019-11-05,413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,2019078601,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-27,2019-12-31,CABUO,U&O Letter,NaT,2020-10-29,True,inspection,2020-10-29,307.0
14156,2019078821,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-30,2019-12-31,CABUO,U&O Letter,NaT,2021-06-24,True,inspection,2021-06-24,542.0
14157,2019079228,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-31,2019-12-31,CABUO,U&O Letter,NaT,2020-09-22,True,inspection,2020-09-22,266.0
14158,2020022071,CACT,Building Commercial - Tenant Finish Out,CAF20M001,"Retail, Department / Retail Stores",DONE,2020-04-07,2020-12-31,CABUO,U&O Letter,2021-03-03,2021-03-18,True,inspection,2021-03-18,345.0


In [14]:
#filter inspection data to include only permits that received Final U&O 
all_depts_inspect = all_depts_inspect[all_depts_inspect['PERMIT #'].isin(u_o['PERMIT #'])]

In [15]:
#combine all data
all_depts_inspect['phase']= 'inspection'
all_depts_app['phase']= 'application'
all_depts = pd.concat([all_depts_app,all_depts_inspect], ignore_index=True)

In [16]:
#create final combined dataframe
amydf = pd.concat([all_depts,u_o])
amydf['deltainspect'] = amydf['Task_Complete_Date'] - amydf['Date_Issued'].copy()
amydf = amydf.astype({'deltainspect':"timedelta64[D]"})

In [17]:
amydf= amydf.reindex(columns=['PERMIT #',
 'CASE_TYPE',
 'CASE_TYPE_DESC',
'dept',
 'SUB_TYPE',
 'SUB_TYPE_DESC',
 'CONSTRUCTION COST',
 'Square Footage',
 'STATUS_CODE',
 'Date_Initiated',
 'Date_Issued',
 'TASK_CODE',
 'TASK_DESC',
 'RESULT_CODE',
 'RESULT_DESC',
 'Task_Complete_Date',
 'deltatask', 
  'deltainspect',
 'postissuedtask',
 'phase',
 'Temp U&O Complete Date',
 'Final U&O Complete Date'])

In [18]:
column = []
for x in amydf['dept']:
    if x is np.nan:
        column.append('All')
    elif x == 'ca':
        column.append('Codes(CA)')
    elif x == 'fr':
        column.append('Fire Marshal(FR)')
    elif x == 'pl':
        column.append('Planning(PL)')
    elif x == 'pw':
        column.append('Public Works(PW)')
    elif x == 'sw':
        column.append('Storm Water (SW)')
    elif x == 'ws':
        column.append('Water Sewer(WS)')
amydf['dept'] = column

In [19]:
amydf

,PERMIT #,CASE_TYPE,CASE_TYPE_DESC,dept,SUB_TYPE,SUB_TYPE_DESC,CONSTRUCTION COST,Square Footage,STATUS_CODE,Date_Initiated,...,TASK_DESC,RESULT_CODE,RESULT_DESC,Task_Complete_Date,deltatask,deltainspect,postissuedtask,phase,Temp U&O Complete Date,Final U&O Complete Date
0,2020055468,CACN,Building Commercial - New,Fire Marshal(FR),CAF21S001,"Self Service Storage, Storage Mod Hazard",727116.00,7800.0,ISSUE,2020-09-03,...,[B] Fire Alarm Requirement,NO,No,2021-05-10,249.0,0.0,False,application,NaT,NaT
1,2020064285,CARN,Building Residential - New,Fire Marshal(FR),CAA01R301,Single Family Residence,226656.08,2062.0,ISSUE,2020-10-13,...,[B] Fire Life Safety Review On Bldg App,IGNORE,No sign-off needed on this project,2021-05-06,205.0,-21.0,False,application,NaT,NaT
2,2020064293,CARN,Building Residential - New,Fire Marshal(FR),CAA01R301,Single Family Residence,226656.08,2062.0,ISSUE,2020-10-13,...,[B] Fire Life Safety Review On Bldg App,IGNORE,No sign-off needed on this project,2021-05-06,205.0,-21.0,False,application,NaT,NaT
3,2020064344,CARN,Building Residential - New,Fire Marshal(FR),CAA01R301,Single Family Residence,226656.08,2062.0,ISSUE,2020-10-13,...,[B] Fire Life Safety Review On Bldg App,IGNORE,No sign-off needed on this project,2021-05-06,205.0,-21.0,False,application,NaT,NaT
4,2020064354,CARN,Building Residential - New,Fire Marshal(FR),CAA01R301,Single Family Residence,226656.08,2062.0,ISSUE,2020-10-13,...,[B] Fire Life Safety Review On Bldg App,IGNORE,No sign-off needed on this project,2021-05-06,205.0,-21.0,False,application,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,2019078601,CARN,Building Residential - New,All,CAA01R301,Single Family Residence,NaN,NaN,DONE,2019-12-27,...,U&O Letter,NaN,NaN,2020-10-29,307.0,303.0,True,inspection,NaT,2020-10-29
14156,2019078821,CARN,Building Residential - New,All,CAA01R301,Single Family Residence,NaN,NaN,DONE,2019-12-30,...,U&O Letter,NaN,NaN,2021-06-24,542.0,541.0,True,inspection,NaT,2021-06-24
14157,2019079228,CARN,Building Residential - New,All,CAA01R301,Single Family Residence,NaN,NaN,DONE,2019-12-31,...,U&O Letter,NaN,NaN,2020-09-22,266.0,266.0,True,inspection,NaT,2020-09-22
14158,2020022071,CACT,Building Commercial - Tenant Finish Out,All,CAF20M001,"Retail, Department / Retail Stores",NaN,NaN,DONE,2020-04-07,...,U&O Letter,NaN,NaN,2021-03-18,345.0,77.0,True,inspection,2021-03-03,2021-03-18


In [20]:
all_pivot_label = all_pivot_label.reindex(columns = ['CASE_TYPE','Case Description', 'DEPARTMENT','MEAN', 'MEDIAN', 'AMIN', 'AMAX', 'STD', 'VAR',
        ])
all_pivot_label

,CASE_TYPE,Case Description,DEPARTMENT,MEAN,MEDIAN,AMIN,AMAX,STD,VAR
0,CACA,Building Commercial - Addition,Fire Marshal(FR),59.356234,30.0,0.0,529.0,80.371588,6459.592161
1,CACF,Building Commercial - Foundation,Fire Marshal(FR),34.920635,15.0,0.0,385.0,58.886835,3467.659286
2,CACG,Building Commercial - Structural Frame,Fire Marshal(FR),58.962264,48.0,16.0,198.0,44.781075,2005.344702
3,CACH,Building Commercial - Shell,Fire Marshal(FR),95.737885,71.0,0.0,604.0,94.584601,8946.246818
4,CACJ,Building Commercial - Fire Damage,Fire Marshal(FR),27.380282,1.0,0.0,152.0,47.071486,2215.724748
...,...,...,...,...,...,...,...,...,...
166,CARP,Building Residential - Change Contractor,All,7.824885,0.0,0.0,124.0,16.301907,265.752174
167,CARR,Building Residential - Rehab,All,5.649149,0.0,-30.0,574.0,18.484825,341.688740
168,CARS,Building Residential New Storm Damage,All,19.288732,10.0,0.0,158.0,26.251133,689.121988
169,CART,Building Residential - Tenant Finish Out,All,66.496190,27.0,0.0,474.0,87.116068,7589.209232


In [21]:
all_pivot_label.to_csv('../data/allpivot.csv', index=False)
all_depts_app.to_csv('../data/all_depts_app.csv', index=False)
all_depts_inspect.to_csv('../data/all_depts_inspect.csv', index=False)
taskfilterdf.to_csv('../data/taskfilter.csv', index=False)
amydf.to_csv('../data/output.csv', index=False)

In [22]:
u_o

,PERMIT #,CASE_TYPE,CASE_TYPE_DESC,SUB_TYPE,SUB_TYPE_DESC,STATUS_CODE,Date_Initiated,Date_Issued,TASK_CODE,TASK_DESC,Temp U&O Complete Date,Final U&O Complete Date,postissuedtask,phase,Task_Complete_Date,deltatask
0,2018000034,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2018-11-26,True,inspection,2018-11-26,328.0
1,2018000187,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2020-05-05,True,inspection,2020-05-05,854.0
2,2018045480,CACN,Building Commercial - New,CAD02B016,"General Office, Professional Services",DONE,2018-08-02,2019-01-02,CABUO,U&O Letter,2020-05-12,2020-07-27,True,inspection,2020-07-27,725.0
3,2018057638,CACR,Building Commercial - Rehab,CAF31A203,Restaurant (Take Out),DONE,2018-09-10,2019-01-02,CABUO,U&O Letter,NaT,2019-11-14,True,inspection,2019-11-14,430.0
4,2018059485,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",DONE,2018-09-18,2019-01-02,CABUO,U&O Letter,NaT,2019-11-05,True,inspection,2019-11-05,413.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,2019078601,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-27,2019-12-31,CABUO,U&O Letter,NaT,2020-10-29,True,inspection,2020-10-29,307.0
14156,2019078821,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-30,2019-12-31,CABUO,U&O Letter,NaT,2021-06-24,True,inspection,2021-06-24,542.0
14157,2019079228,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-31,2019-12-31,CABUO,U&O Letter,NaT,2020-09-22,True,inspection,2020-09-22,266.0
14158,2020022071,CACT,Building Commercial - Tenant Finish Out,CAF20M001,"Retail, Department / Retail Stores",DONE,2020-04-07,2020-12-31,CABUO,U&O Letter,2021-03-03,2021-03-18,True,inspection,2021-03-18,345.0


In [23]:
u_o['deltaapp'] = u_o['Date_Issued'] - u_o['Date_Initiated'].copy()
u_o['deltainspect'] = u_o['Task_Complete_Date'] - u_o['Date_Issued'].copy()
u_o = u_o.astype({'deltaapp':"timedelta64[D]"})
u_o = u_o.astype({'deltainspect':"timedelta64[D]"})
u_o

,PERMIT #,CASE_TYPE,CASE_TYPE_DESC,SUB_TYPE,SUB_TYPE_DESC,STATUS_CODE,Date_Initiated,Date_Issued,TASK_CODE,TASK_DESC,Temp U&O Complete Date,Final U&O Complete Date,postissuedtask,phase,Task_Complete_Date,deltatask,deltaapp,deltainspect
0,2018000034,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2018-11-26,True,inspection,2018-11-26,328.0,0.0,328.0
1,2018000187,CARR,Building Residential - Rehab,CAA01R301,Single Family Residence,DONE,2018-01-02,2018-01-02,CABUO,U&O Letter,NaT,2020-05-05,True,inspection,2020-05-05,854.0,0.0,854.0
2,2018045480,CACN,Building Commercial - New,CAD02B016,"General Office, Professional Services",DONE,2018-08-02,2019-01-02,CABUO,U&O Letter,2020-05-12,2020-07-27,True,inspection,2020-07-27,725.0,153.0,572.0
3,2018057638,CACR,Building Commercial - Rehab,CAF31A203,Restaurant (Take Out),DONE,2018-09-10,2019-01-02,CABUO,U&O Letter,NaT,2019-11-14,True,inspection,2019-11-14,430.0,114.0,316.0
4,2018059485,CACR,Building Commercial - Rehab,CAD02B016,"General Office, Professional Services",DONE,2018-09-18,2019-01-02,CABUO,U&O Letter,NaT,2019-11-05,True,inspection,2019-11-05,413.0,106.0,307.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,2019078601,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-27,2019-12-31,CABUO,U&O Letter,NaT,2020-10-29,True,inspection,2020-10-29,307.0,4.0,303.0
14156,2019078821,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-30,2019-12-31,CABUO,U&O Letter,NaT,2021-06-24,True,inspection,2021-06-24,542.0,1.0,541.0
14157,2019079228,CARN,Building Residential - New,CAA01R301,Single Family Residence,DONE,2019-12-31,2019-12-31,CABUO,U&O Letter,NaT,2020-09-22,True,inspection,2020-09-22,266.0,0.0,266.0
14158,2020022071,CACT,Building Commercial - Tenant Finish Out,CAF20M001,"Retail, Department / Retail Stores",DONE,2020-04-07,2020-12-31,CABUO,U&O Letter,2021-03-03,2021-03-18,True,inspection,2021-03-18,345.0,268.0,77.0


In [24]:
u_o_pivot = pd.pivot_table(u_o, values=['deltaapp', 'deltainspect', 'deltatask'], index='CASE_TYPE_DESC', aggfunc=np.mean)

In [25]:
u_o_pivot.sort_values('deltaapp', ascending=False)

,deltaapp,deltainspect,deltatask
CASE_TYPE_DESC,,,
Building Residential - Tenant Finish Out,192.727273,517.977273,710.704545
Building Residential - Shell,147.000000,592.000000,739.000000
Building Commercial - Shell,138.305085,481.016949,619.322034
Building Commercial - New,122.548638,407.354086,529.902724
Building Commercial - Tenant Finish Out,98.873724,307.094388,405.968112
Building Commercial - Structural Frame,92.000000,745.500000,837.500000
Building Commercial - Addition,86.153846,259.707692,345.861538
Building Commercial - Rehab,59.180419,197.619458,256.799877
Building Commercial - Foundation,48.363636,553.363636,601.727273


In [26]:
new_columns = {'deltaapp': 'Application Phase',
               'deltainspect' : 'Inspection Phase', 
               'deltatask': 'Total'}

u_o_pivot = u_o_pivot.reset_index()
u_o_pivot.columns = u_o_pivot.columns.map(''.join)
u_o_pivot.rename(columns = new_columns, inplace=True)
u_o_pivot.columns

Index(['CASE_TYPE_DESC', 'Application Phase', 'Inspection Phase', 'Total'], dtype='object')

In [27]:
u_o_pivot

,CASE_TYPE_DESC,Application Phase,Inspection Phase,Total
0,Building Commercial - Addition,86.153846,259.707692,345.861538
1,Building Commercial - Change Contractor,10.000000,145.333333,155.333333
2,Building Commercial - Fire Damage,43.611111,304.555556,348.166667
3,Building Commercial - Foundation,48.363636,553.363636,601.727273
4,Building Commercial - New,122.548638,407.354086,529.902724
5,Building Commercial - Rehab,59.180419,197.619458,256.799877
6,Building Commercial - Roofing / Siding,6.285714,248.428571,254.714286
7,Building Commercial - Shell,138.305085,481.016949,619.322034
8,Building Commercial - Structural Frame,92.000000,745.500000,837.500000
9,Building Commercial - Tenant Finish Out,98.873724,307.094388,405.968112


In [28]:
u_o_pivot.sort_values('Application Phase', ascending=False).to_csv('../data/averagetimes.csv', index=False)